In [16]:
import cv2
import pyAprilTag
import numpy as np
import matplotlib.pyplot as plt

In [24]:
def draw_cube(image,new_points,corners):     #draw cube on image 
    cp = corners.astype(int)
    np = new_points.astype(int)
    for i in range(4):                     #connect side points
        x1,y1 = cp[i,0],cp[i,1]
        x2,y2 = np[i,0],np[i,1]
        cv2.line(image, (x1,y1),(x2,y2), (0,0,255), 2)
    for i in range(-1,3):                  #connect bottom points
        x1,y1 = cp[i,0],cp[i,1]
        x2,y2 = cp[i+1,0],cp[i+1,1]
        cv2.line(image, (x1,y1),(x2,y2), (0,0,255), 2)
    for i in range(-1,3):                    #connect top points
        x1,y1 = np[i,0],np[i,1]
        x2,y2 = np[i+1,0],np[i+1,1]
        cv2.line(image, (x1,y1),(x2,y2), (0,0,255), 2)
    return image

In [29]:
im = cv2.imread("frame3.png")
img = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
K=np.array([[816.2117515245018, 0, 310.2413717879887],
           [0, 810.9569296724389, 254.8559962566917],
           [0, 0, 1]], dtype='float64')

distCoeffs=np.array([0.04076376547927418,
       0.2748835592290237,
       0.002470542038679734,
       0.0006316162578014795,
       0], dtype='float64')

ids, corners, centers, Hs = pyAprilTag.find(img)

# #PNP needs atleast 6 points correspondences
corners = corners.reshape(4,2)
centers = centers.reshape(1,2)
x = (corners[0,0] + corners[1,0])/2      #take first two coordinates and find the center of them
y = (corners[0,1] + corners[1,1])/2
last_point = np.array([[x,y]]).reshape(1,2)

im_points = np.vstack((corners,centers,last_point))

world_points = np.array([[0.0,0.0,0.0],      #must be in float form
                         [0.0,-11.0,0.0],    # world points got from the actual measurement of the size of the april tag
                         [11.0,-11.0,0.0],
                         [11.0,0.0,0.0],
                         [5.5,-5.5,0],     
                         [0.0,-5.5,0]],dtype="float32")
(state,Rt,T) = cv2.solvePnP(world_points,im_points,K,distCoeffs) # solve Pnp

w_o = 11    #offset in the z direction to create a cube
cube = np.array([[0.0,0.0,0.0+w_o],     
                 [0.0,-11.0,0.0+w_o],
                 [11.0,-11.0,0.0+w_o],
                 [11.0,0.0,0.0+w_o]],dtype="float32")
Projected_points,jacobian = cv2.projectPoints(cube,Rt,T,K,distCoeffs) #get new projected points
new_points = Projected_points.reshape(len(Projected_points),2)
image = draw_cube(im,corners,new_points)
cv2.imshow("image",image)
cv2.waitKey(0)

-1

# Real Time cube construction

In [26]:
camid = 1
cap = cv2.VideoCapture(camid)

if not cap.isOpened():
    print("Error opening video stream or file")
    exit(0)

K=np.array([[816.2117515245018, 0, 310.2413717879887],
           [0, 810.9569296724389, 254.8559962566917],
           [0, 0, 1]], dtype='float64')

distCoeffs=np.array([0.04076376547927418,
       0.2748835592290237,
       0.002470542038679734,
       0.0006316162578014795,
       0], dtype='float64')

world_points = np.array([[0.0,0.0,0.0],      #must be in float form
                         [0.0,-11.0,0.0],
                         [11.0,-11.0,0.0],
                         [11.0,0.0,0.0],
                         [5.5,-5.5,0],     
                         [0.0,-5.5,0]],dtype="float32")

w_o = 11    
cube = np.array([[0.0,0.0,0.0+w_o],     
                 [0.0,-11.0,0.0+w_o],
                 [11.0,-11.0,0.0+w_o],
                 [11.0,0.0,0.0+w_o]],dtype="float32")

last_point = np.array([[0,0]])
im_points = np.zeros((6,2),dtype="float32")
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break
    
    ids, corners, centers, Hs = pyAprilTag.find(frame)
    

    
    if len(corners) > 0:
        corners = corners.reshape(4,2)
        centers = centers.reshape(1,2)
        x = (corners[0,0] + corners[1,0])/2      #take first two coordinates and find the center of them
        y = (corners[0,1] + corners[1,1])/2
        
        last_point[0,0]= x
        last_point[0,1]= y
        
        im_points[:4:] = corners
        im_points[4:5:] = centers
        im_points[5:6:] = last_point 
        
        (state,Rt,T) = cv2.solvePnP(world_points,im_points,K,distCoeffs)

        Projected_points,jacobian = cv2.projectPoints(cube,Rt,T,K,distCoeffs)
        new_points = Projected_points.reshape(len(Projected_points),2)
        
        frame = draw_cube(frame,new_points,corners)

    # Display the resulting frame
    cv2.imshow('Frame',frame)
    # Press ESC on keyboard to exit
    
    if cv2.waitKey(1) & 0XFF ==ord("q"):
        break
      

#When everything done, release the video capture object
cap.release()
# Closes all the frames
cv2.destroyAllWindows()